In [2]:
import pandas as pd

In [ ]:
df1= pd.read_json('data/gdpr/articles/gdpr_articles_recitals.jsonl', lines=True)  
df2= pd.read_csv('data/gdpr/articles/GDPR_10QA_dataset_filtered.csv')
df3= pd.read_csv('data/gdpr/articles/qa_GDPR_dataset.csv')

# Dataset 1: GDPR Article Number+Title - Entire Article

In [57]:
df1.head()

,input-text,output-text
0,Article 1 Subject-matter and objectives,(1) This Regulation lays down rules relating t...
1,Article 2 Material scope,(1) This Regulation applies to the processing ...
2,Article 3 Territorial scope,(1) This Regulation applies to the processing ...
3,Article 4 Definitions,For the purposes of this Regulation: (1) ‘pers...
4,Article 5 Principles relating to processing of...,(1) Personal data shall be:\n \n- proce...


**format to { "text": article/recital + : content}**

In [ ]:
df1['output-text'] = df1['output-text'].str.replace("\n", " ", regex=False)

df1['text'] = df1.apply(lambda row: f'{{"text": "{row["input-text"]}: {row["output-text"]}"}}', axis=1)

with open('data/gdpr/articles/preprocessed/gdpr_articles_recitals_preprocessed.jsonl', 'w') as file:
    for item in df1['text']:
        file.write(item + '\n')

print("JSONL file has been created with formatted text.")

JSONL file has been created with formatted text.


# Dataset 2: GDPR csv

Article Number, Article Name, Chapter Numbers, Chapter Names, Information 

In [59]:
df2.head()

,Content,Article Number,Article Name,Chapter Number,Chapter Name,Article Word Count,Question,Answer,Question Word Count,Answer Word Count
0,\nArticle 1\nSubject-matter and objectives\n\n...,1,Subject-matter and objectives,1,General provisions,91,What is the purpose of GDPR?,The purpose of GDPR is to lay down rules relat...,6,34
1,\nArticle 2\nMaterial scope\n\n1. This Regulat...,2,Material scope,1,General provisions,232,What is the scope of GDPR?,GDPR applies to the processing of personal dat...,6,43
2,\nArticle 3\nTerritorial scope\n\n1. This Regu...,3,Territorial scope,1,General provisions,154,What is the territorial scope of GDPR?,GDPR applies to the processing of personal dat...,7,38
3,\nArticle 4\nDefinitions\n\nFor the purposes o...,4,Definitions,1,General provisions,935,What is the definition of personal data accord...,Personal data means any information relating t...,10,69
4,\nArticle 4 (continued)\nDefinitions\n\nFor th...,4,Definitions,1,General provisions,405,What is the definition of 'group of undertakin...,"For the purposes of this Regulation, 'group of...",11,17


In [60]:
df2.columns.values

array(['Content', 'Article Number', 'Article Name', 'Chapter Number',
       'Chapter Name', 'Article Word Count', 'Question', 'Answer',
       'Question Word Count', 'Answer Word Count'], dtype=object)

In [61]:
df3_QA= df2[['Article Number', 'Article Name', 'Chapter Number', 'Chapter Name', 'Question', 'Answer']]
df2=df2[['Article Number', 'Article Name', 'Chapter Number', 'Chapter Name', 'Content']]

In [ ]:
df2['Content'] = df2['Content'].str.replace("\n", " ", regex=False)

df2['text'] = df2.apply(lambda row: f'{{"text": "Article {row["Article Number"]} {row["Article Name"]}, '
                                    f'Chapter {row["Chapter Number"]} {row["Chapter Name"]}: {row["Content"]}"}}', axis=1)

with open('data/gdpr/articles/preprocessed/gdpr_article_chapters_content_preprocessed.jsonl', 'w') as file:
    for item in df2['text']:
        file.write(item + '\n')

print("JSONL file has been created with formatted text.")


JSONL file has been created with formatted text.


# Dataset 3: QA

In [63]:
df3_QA['text'] = df3_QA.apply(lambda row: f'{{"question": "{row["Question"]} reference Article {row["Article Number"]} {row["Article Name"]}, '
                                    f'Chapter {row["Chapter Number"]} {row["Chapter Name"]}", "answer": "{row["Answer"]} "}}', axis=1)

In [64]:
df3['text'] = df3.apply(lambda row: f'{{"question": "{row["Question"]}", "answer": "{row["Correct Answer"]} "}}', axis=1)

In [ ]:
file_path = 'data/gdpr/articles/preprocessed/gdpr_QA_preprocessed.jsonl'

with open(file_path, 'w') as file:
    for item in df3_QA['text']:
        file.write(item + '\n')

with open(file_path, 'a') as file:  
    for item in df3['text']:
        file.write(item + '\n')

print("JSONL file has been updated with formatted text from both dataframes.")


JSONL file has been updated with formatted text from both dataframes.


# Dataset QA to knowedle text dataset

In [ ]:
dfqa= pd.read_json('data/gdpr/articles/preprocessed/gdpr_QA_preprocessed.jsonl', lines=True)
dfqa['text'] = dfqa['answer']
contexts = dfqa['text'].apply(lambda x: x.split('reference')[0].strip())

In [ ]:
import json

input_path = "data/gdpr/articles/preprocessed/gdpr_QA_preprocessed.jsonl"
output_path = "..ata/gdpr/articles/preprocessed/gdpr_QA_to_Text_preprocessed.jsonl"

with open(input_path, "r") as infile, open(output_path, "w") as outfile:
    for line in infile:
        data = json.loads(line)
        
        answer = data.get("answer", "")
        
        question = data.get("question", "")
        if "reference" in question:
            reference = question.split("reference", 1)[1].strip()
        else:
            reference = "No reference"
        
        combined_text = f"{answer} Reference : {reference}"
        
        new_data = {"text": combined_text}
        
        outfile.write(json.dumps(new_data) + "\n")

print("New JSONL file created successfully!")


New JSONL file created successfully!
